In [1]:
import geopandas as gpd
import pandas as pd
import numpy as np

import sklearn
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import make_pipeline

# Chargement des données CSV

In [2]:
lieux_df = pd.read_csv('../clean_data/lieux_clean.csv')
usagers_df = pd.read_csv('../clean_data/usagers_clean.csv')
vehicules_df = pd.read_csv('../clean_data/vehicules_clean.csv')
caracteristiques_df = pd.read_csv('../clean_data/caracteristiques_clean.csv')

/var/folders/jw/qw353j7x7h970k1t74h_j5nc0000gn/T/ipykernel_23082/4187407229.py:4: DtypeWarning: Columns (6,8,9,10,11) have mixed types. Specify dtype option on import or set low_memory=False.
  caracteristiques_df = pd.read_csv('../clean_data/caracteristiques_clean.csv')


In [3]:
vehicules_df

,Num_Acc,catv,obs,obsm,choc,manv,num_veh
0,201800000001,7,0.0,2.0,3.0,1.0,B01
1,201800000001,7,0.0,2.0,2.0,15.0,A01
2,201800000002,7,0.0,1.0,1.0,1.0,A01
3,201800000003,33,1.0,2.0,1.0,1.0,A01
4,201800000003,7,0.0,2.0,8.0,15.0,B01
...,...,...,...,...,...,...,...
2009390,201500058652,7,0.0,2.0,1.0,17.0,A01
2009391,201500058652,30,0.0,2.0,1.0,1.0,B01
2009392,201500058653,7,0.0,1.0,2.0,1.0,A01
2009393,201500058654,30,0.0,2.0,1.0,1.0,A01


In [4]:
filtered_df = vehicules_df[vehicules_df['catv'] == 1]


In [5]:
duplicates = filtered_df[filtered_df['Num_Acc'].duplicated(keep=False)]
print(duplicates)


              Num_Acc  catv   obs  obsm  choc  manv num_veh
346      201800000213     1   0.0   2.0   4.0   1.0     B01
347      201800000213     1   0.0   9.0   1.0   1.0     C01
359      201800000222     1   0.0   2.0   1.0   1.0     A01
360      201800000222     1   0.0   0.0   4.0   2.0     B01
1678     201800001030     1   0.0   2.0   8.0   1.0     B01
...               ...   ...   ...   ...   ...   ...     ...
2004463  201500055887     1   0.0   0.0   1.0  19.0     B01
2006753  201500057100     1  13.0   0.0   1.0   1.0     B02
2006754  201500057100     1  13.0   0.0   1.0   1.0     A01
2009282  201500058584     1   0.0   9.0   1.0   2.0     A01
2009283  201500058584     1   0.0   9.0   4.0   1.0     B01

[5399 rows x 7 columns]


In [6]:
caracteristiques_df.isna().sum()

caracteristiques_df[caracteristiques_df['Num_Acc'].isna()]

,Num_Acc,lum,agg,int,atm,col,com,adr,lat,long,dep,date
116623,NaN,1,2,3,1.0,3.0,26198,TEIL(vieille route du),"44,5594200000","4,7257200000",26,NaN
116624,NaN,1,2,3,1.0,3.0,25204,Miranda,"46,9258100000","6,3462000000",25,NaN
116625,NaN,1,2,6,1.0,2.0,22360,ROND POINT DE BREZILLET,"48,4931620000","-2,7604390000",22,NaN
116626,NaN,1,2,3,8.0,6.0,16102,LOHMEYER (RUE),"45,6926520000","-0,3262900000",16,NaN
116627,NaN,1,1,1,1.0,2.0,13103,ROUTE DE JEAN MOULIN-RN 538,"43,6755790366","5,0927031775",13,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
171920,NaN,3,1,1,1.0,6.0,2B293,D71,"42,3101650000","9,4785830000",2B,NaN
171921,NaN,3,1,1,1.0,6.0,84074,D973,"43,7531640000","5,2244760000",84,NaN
171922,NaN,3,1,1,9.0,6.0,74001,D22,"46,2825320000","6,7328060000",74,NaN
171923,NaN,1,1,3,1.0,3.0,81099,Chemin Toulze,"43,9272650000","1,9156370000",81,NaN


In [7]:
# fonction rolliwn window ? applique une fonction sur une partie précise


# Merge de la données


In [8]:
# Création d'une colonne uniques pour merger les df usagers et véhicules

usagers_df['aug'] = usagers_df['Num_Acc'].astype(str) + usagers_df['num_veh'].astype(str)
vehicules_df['aug'] = vehicules_df['Num_Acc'].astype(str) + vehicules_df['num_veh'].astype(str)


In [9]:
vehicules_df.shape

(2009395, 8)

In [10]:
vehicules_df.shape
vehicules_df = vehicules_df[vehicules_df['catv']==1]

In [11]:
# On drop les duplicates lorsqu'il y a une plusieurs usagers dans un même véhicules et pour le même accident
# On crée une colonne pour y ajouter le nombre d'usagers par véhicules


usagers_df['dup_count'] = usagers_df.groupby('aug')['aug'].transform('count')
usagers_df =usagers_df.drop_duplicates(subset=['aug'], keep='first')

In [12]:
print(type(vehicules_df))
print(type(usagers_df))

<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>


In [13]:
# On merge véhicules et usagers puis on ne garde que les accidents impliquant un vélo
# On clean et on drop les doublons

vehi_usa = pd.merge(vehicules_df, usagers_df, on='aug', how='inner')
vehi_usa = vehi_usa.drop(columns=['num_veh_x', 'num_veh_y', 'Num_Acc_y'])
vehi_usa = vehi_usa.rename(columns={'Num_Acc_x': 'Num_Acc'})


In [14]:
vehi_usa['dup_count'].value_counts()

1    83223
2     5132
3      156
4        5
6        3
5        2
Name: dup_count, dtype: int64

In [15]:
print(vehi_usa['Num_Acc'].duplicated().any())
print(caracteristiques_df['Num_Acc'].duplicated().any())


True
True


In [16]:
duplicates_velo = vehi_usa[vehi_usa['Num_Acc'].duplicated(keep=False)]
duplicates_velo.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5148 entries, 12 to 88515
Data columns (total 16 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Num_Acc    5148 non-null   int64  
 1   catv       5148 non-null   int64  
 2   obs        5148 non-null   float64
 3   obsm       5148 non-null   float64
 4   choc       5148 non-null   float64
 5   manv       5148 non-null   float64
 6   aug        5148 non-null   object 
 7   grav       5148 non-null   int64  
 8   sexe       5148 non-null   int64  
 9   trajet     5148 non-null   int64  
 10  secu       5148 non-null   float64
 11  locp       5148 non-null   float64
 12  actp       5148 non-null   int64  
 13  etatp      5148 non-null   float64
 14  an_nais    5148 non-null   float64
 15  dup_count  5148 non-null   int64  
dtypes: float64(8), int64(7), object(1)
memory usage: 683.7+ KB


In [17]:
# On merge les df caractéristiques et lieux

velo_df = pd.merge(vehi_usa, lieux_df, on='Num_Acc', how='left')
velo_df = pd.merge(velo_df, caracteristiques_df, on='Num_Acc', how='left')


In [18]:
velo_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 88521 entries, 0 to 88520
Data columns (total 36 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Num_Acc    88521 non-null  int64  
 1   catv       88521 non-null  int64  
 2   obs        88521 non-null  float64
 3   obsm       88521 non-null  float64
 4   choc       88521 non-null  float64
 5   manv       88521 non-null  float64
 6   aug        88521 non-null  object 
 7   grav       88521 non-null  int64  
 8   sexe       88521 non-null  int64  
 9   trajet     88521 non-null  int64  
 10  secu       88521 non-null  float64
 11  locp       88521 non-null  float64
 12  actp       88521 non-null  int64  
 13  etatp      88521 non-null  float64
 14  an_nais    88521 non-null  float64
 15  dup_count  88521 non-null  int64  
 16  catr       88521 non-null  float64
 17  circ       88521 non-null  float64
 18  nbv        88521 non-null  float64
 19  vosp       88521 non-null  float64
 20  prof  

In [19]:
velo_df.head()

,Num_Acc,catv,obs,obsm,choc,manv,aug,grav,sexe,trajet,...,agg,int,atm,col,com,adr,lat,long,dep,date
0,201800000004,1,0.0,2.0,8.0,1.0,201800000004B01,3,1,5,...,2.0,1.0,7.0,3.0,52,30 rue Jules Guesde,5051974.0,289123.0,590,2018-05-05 17:35:00
1,201800000020,1,12.0,0.0,7.0,1.0,201800000020B01,3,1,9,...,1.0,1.0,1.0,7.0,586,rue de wachemy,5054129.0,318902.0,590,2018-05-27 16:00:00
2,201800000044,1,0.0,0.0,8.0,2.0,201800000044A01,3,1,5,...,1.0,1.0,7.0,3.0,400,rue de maroeuil,5062556.0,262903.0,590,2018-09-29 18:00:00
3,201800000051,1,0.0,0.0,4.0,2.0,201800000051B01,2,1,5,...,2.0,1.0,2.0,2.0,42,Rte départementale,5053908.0,326460.0,590,2018-08-26 15:30:00
4,201800000056,1,0.0,2.0,6.0,2.0,201800000056B01,3,1,5,...,1.0,1.0,1.0,2.0,252,105 rue de quesnoy sur d,5068934.0,296942.0,590,2018-04-10 15:20:00


In [20]:
# On remplace les valeurs manquantes par la valeur la plus fréquente
velo_df['lum'].fillna(velo_df['lum'].mode()[0], inplace=True)
velo_df['agg'].fillna(velo_df['agg'].mode()[0], inplace=True)
velo_df['atm'].fillna(velo_df['atm'].mode()[0], inplace=True)
velo_df['atm'] = velo_df['atm'].replace({ -1: 1, 9: 1})

In [21]:
# Obtenez la distribution des valeurs
distribution = velo_df['int'].value_counts(normalize=True)
# Obtenez les valeurs manquantes
missing = velo_df['int'].isnull()
# Remplissez les valeurs manquantes en fonction de la distribution
velo_df.loc[missing, 'int'] = np.random.choice(distribution.index, size=len(velo_df[missing]), p=distribution.values)

In [22]:
# Obtenez la distribution des valeurs
distribution = velo_df['col'].value_counts(normalize=True)
# Obtenez les valeurs manquantes
missing = velo_df['col'].isnull()
# Remplissez les valeurs manquantes en fonction de la distribution
velo_df.loc[missing, 'col'] = np.random.choice(distribution.index, size=len(velo_df[missing]), p=distribution.values)
# Je remplace le -1 par 1
velo_df['col'] = velo_df['col'].replace({ -1: 3})

In [23]:
velo_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 88521 entries, 0 to 88520
Data columns (total 36 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Num_Acc    88521 non-null  int64  
 1   catv       88521 non-null  int64  
 2   obs        88521 non-null  float64
 3   obsm       88521 non-null  float64
 4   choc       88521 non-null  float64
 5   manv       88521 non-null  float64
 6   aug        88521 non-null  object 
 7   grav       88521 non-null  int64  
 8   sexe       88521 non-null  int64  
 9   trajet     88521 non-null  int64  
 10  secu       88521 non-null  float64
 11  locp       88521 non-null  float64
 12  actp       88521 non-null  int64  
 13  etatp      88521 non-null  float64
 14  an_nais    88521 non-null  float64
 15  dup_count  88521 non-null  int64  
 16  catr       88521 non-null  float64
 17  circ       88521 non-null  float64
 18  nbv        88521 non-null  float64
 19  vosp       88521 non-null  float64
 20  prof  

In [24]:
velo_df = velo_df[~(velo_df.adr.isna() & velo_df.lat.isna() & velo_df.long.isna() & velo_df.com.isna() & velo_df.dep.isna())]

In [25]:
# Réinitialisez l'index de votre DataFrame
velo_df = velo_df.reset_index(drop=True)

# Obtenez la distribution des valeurs
distribution = velo_df['date'].value_counts(normalize=True)

# Obtenez les valeurs manquantes
missing = velo_df['date'].isnull()

# Remplissez les valeurs manquantes en fonction de la distribution
velo_df.loc[missing, 'date'] = np.random.choice(distribution.index, size=len(velo_df[missing]), p=distribution.values)

In [26]:
velo_df.drop(columns=['catv'], inplace=True)


In [27]:
velo_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83233 entries, 0 to 83232
Data columns (total 35 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Num_Acc    83233 non-null  int64  
 1   obs        83233 non-null  float64
 2   obsm       83233 non-null  float64
 3   choc       83233 non-null  float64
 4   manv       83233 non-null  float64
 5   aug        83233 non-null  object 
 6   grav       83233 non-null  int64  
 7   sexe       83233 non-null  int64  
 8   trajet     83233 non-null  int64  
 9   secu       83233 non-null  float64
 10  locp       83233 non-null  float64
 11  actp       83233 non-null  int64  
 12  etatp      83233 non-null  float64
 13  an_nais    83233 non-null  float64
 14  dup_count  83233 non-null  int64  
 15  catr       83233 non-null  float64
 16  circ       83233 non-null  float64
 17  nbv        83233 non-null  float64
 18  vosp       83233 non-null  float64
 19  prof       83233 non-null  float64
 20  plan  

In [28]:
print(velo_df['com'])

0         52
1        586
2        400
3         42
4        252
        ... 
83228    611
83229    611
83230    611
83231    611
83232    611
Name: com, Length: 83233, dtype: object


In [29]:
velo_df.to_csv('../clean_data/velo_df.csv', index=False)

# preprocess

In [30]:
from sklearn.preprocessing import OneHotEncoder, FunctionTransformer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_transformer
import datetime as dt


In [37]:
def calculate_age(df):
            # Extrait les 4 premiers chiffres de 'Num_Acc' comme année de l'accident
            df['annee_accident'] = df['Num_Acc'].astype(str).str[:4]

            # Convertit 'annee_accident' en type entier
            df['annee_accident'] = df['annee_accident'].astype(int)

            # Calcul de l'âge à la date de l'accident
            df['age_at_accident'] = df['annee_accident'] - df['an_nais']

            # Supprime les colonnes intermédiaires si nécessaire
            df = df.drop(['annee_accident'], axis=1)

            return df

In [38]:
calculate_age(velo_df)

,Num_Acc,obs,obsm,choc,manv,aug,grav,sexe,trajet,secu,...,int,atm,col,com,adr,lat,long,dep,date,age_at_accident
0,201800000004,0.0,2.0,8.0,1.0,201800000004B01,3,1,5,1.0,...,1.0,7.0,3.0,52,30 rue Jules Guesde,5051974.0,289123.0,590,2018-05-05 17:35:00,5.0
1,201800000020,12.0,0.0,7.0,1.0,201800000020B01,3,1,9,1.0,...,1.0,1.0,7.0,586,rue de wachemy,5054129.0,318902.0,590,2018-05-27 16:00:00,21.0
2,201800000044,0.0,0.0,8.0,2.0,201800000044A01,3,1,5,1.0,...,1.0,7.0,3.0,400,rue de maroeuil,5062556.0,262903.0,590,2018-09-29 18:00:00,71.0
3,201800000051,0.0,0.0,4.0,2.0,201800000051B01,2,1,5,1.0,...,1.0,2.0,2.0,42,Rte départementale,5053908.0,326460.0,590,2018-08-26 15:30:00,78.0
4,201800000056,0.0,2.0,6.0,2.0,201800000056B01,3,1,5,1.0,...,1.0,1.0,2.0,252,105 rue de quesnoy sur d,5068934.0,296942.0,590,2018-04-10 15:20:00,74.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83228,201500058611,14.0,2.0,1.0,17.0,201500058611B01,4,1,5,1.0,...,1.0,1.0,3.0,611,mamoudzou centre-Préfect,0.0,0.0,976,2015-11-05 16:50:00,55.0
83229,201500058616,0.0,2.0,1.0,1.0,201500058616B01,3,1,1,1.0,...,6.0,1.0,3.0,611,doujani-Mosquée doujani,0.0,0.0,976,2015-09-28 16:50:00,51.0
83230,201500058626,0.0,1.0,1.0,2.0,201500058626A01,1,1,9,1.0,...,1.0,1.0,6.0,611,m'tsapere-Dahilou (rue),0.0,0.0,976,2015-10-07 14:53:00,13.0
83231,201500058629,1.0,0.0,1.0,15.0,201500058629A01,4,1,5,1.0,...,4.0,1.0,3.0,611,mamoudzou centre-Préfect,0.0,0.0,976,2015-10-13 11:35:00,13.0


In [81]:
import pandas as pd
import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, FunctionTransformer
from sklearn.pipeline import make_pipeline

def preprocess_features(X):

    def create_sklearn_preprocessor() -> ColumnTransformer:
        ohe_categories = ['obs', 'obsm', 'choc', 'manv', 'trajet', 'locp', 'actp', 'etatp', 'catr', 'circ', 'vosp',
                          'prof', 'plan', 'surf', 'infra', 'situ', 'lum', 'int', 'col']

        ohe_pipe = ColumnTransformer(
            transformers=[
                ('ohe', OneHotEncoder(categories='auto', sparse_output=False, handle_unknown='ignore'), ohe_categories)
            ],
            remainder='passthrough'
        )

        grav_pipe = ColumnTransformer(
            transformers=[
                ('grav', FunctionTransformer(lambda x: np.where(x == 1, 'Indemne',
                                                                 np.where(x == 2, 'Blessé léger',
                                                                          np.where(x == 3, 'Blessé hospitalisé',
                                                                                   np.where(x == 4, 'Tué', x))))), [X.columns.get_loc('grav')],)
            ], remainder='passthrough'
        )
            # Remplace les valeurs 2 par 0 dans les colonnes spécifiées
        X['sexe'] = np.where(X['sexe'] == 2, 0, X['sexe'])
        X['secu'] = np.where(X['secu'] == 2, 0, X['secu'])
        X['agg'] = np.where(X['agg'] == 2, 0, X['agg'])

        # Sélectionne les colonnes à traiter dans le reste du pipeline
        columns_to_pass_through = [col for col in X.columns if col not in ['sexe', 'secu', 'agg']]

        passthrough_pipe = ColumnTransformer(
            transformers=[
                ('passthrough', 'passthrough', columns_to_pass_through)
            ], remainder='passthrough'
        )

        passthrough_pipe = ColumnTransformer(
            transformers=[
                ('passthrough', 'passthrough', np.arange(X.shape[1]))  # Pass indices instead of column names
            ], remainder='passthrough'
        )

        final_pipeline = make_pipeline(
            ohe_pipe,
            grav_pipe,
            passthrough_pipe
        )

        return final_pipeline

    # Assurez-vous que X est un DataFrame pandas
    if not isinstance(X, pd.DataFrame):
        X = pd.DataFrame(X)

    preprocessor = create_sklearn_preprocessor()

    X_processed = preprocessor.fit_transform(X)

    X_processed = pd.DataFrame(X_processed)

    print("✅ X_processed, with shape", X_processed.shape)

    return X_processed

# Exemple d'utilisation
X_process = preprocess_features(X)


✅ X_processed, with shape (83233, 170)


In [85]:
X = velo_df.drop(columns=['Num_Acc', 'aug', 'com', 'adr', 'lat', 'long', 'dep', 'annee_accident', 'an_nais'])
velo_df.head()

,Num_Acc,obs,obsm,choc,manv,aug,grav,sexe,trajet,secu,...,atm,col,com,adr,lat,long,dep,date,annee_accident,age_at_accident
0,201800000004,0.0,2.0,8.0,1.0,201800000004B01,3,1,5,1.0,...,7.0,3.0,52,30 rue Jules Guesde,5051974.0,289123.0,590,2018-05-05 17:35:00,2018,5.0
1,201800000020,12.0,0.0,7.0,1.0,201800000020B01,3,1,9,1.0,...,1.0,7.0,586,rue de wachemy,5054129.0,318902.0,590,2018-05-27 16:00:00,2018,21.0
2,201800000044,0.0,0.0,8.0,2.0,201800000044A01,3,1,5,1.0,...,7.0,3.0,400,rue de maroeuil,5062556.0,262903.0,590,2018-09-29 18:00:00,2018,71.0
3,201800000051,0.0,0.0,4.0,2.0,201800000051B01,2,1,5,1.0,...,2.0,2.0,42,Rte départementale,5053908.0,326460.0,590,2018-08-26 15:30:00,2018,78.0
4,201800000056,0.0,2.0,6.0,2.0,201800000056B01,3,1,5,1.0,...,1.0,2.0,252,105 rue de quesnoy sur d,5068934.0,296942.0,590,2018-04-10 15:20:00,2018,74.0


In [86]:
X_process.head()

,0,1,2,3,4,5,6,7,8,9,...,160,161,162,163,164,165,166,167,168,169
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,3.0,1.0,1.0,2013.0,1.0,2.0,0.0,7.0,2018.0,5.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,3.0,1.0,1.0,1997.0,1.0,2.0,1.0,1.0,2018.0,21.0
2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,3.0,1.0,1.0,1947.0,1.0,2.0,1.0,7.0,2018.0,71.0
3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.0,1.0,1.0,1940.0,1.0,1.0,0.0,2.0,2018.0,78.0
4,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,3.0,1.0,1.0,1944.0,1.0,2.0,1.0,1.0,2018.0,74.0
